# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'04-01-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'04-01-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-04-02 04:20:36,33.93911,67.709953,56517,2489,51788.0,2240.0,Afghanistan,145.182144,4.403985
1,NaN,NaN,NaN,Albania,2021-04-02 04:20:36,41.15330,20.168300,125506,2241,91875.0,31390.0,Albania,4361.178678,1.785572
2,NaN,NaN,NaN,Algeria,2021-04-02 04:20:36,28.03390,1.659600,117304,3096,81632.0,32576.0,Algeria,267.505610,2.639296
3,NaN,NaN,NaN,Andorra,2021-04-02 04:20:36,42.50630,1.521800,12053,115,11365.0,573.0,Andorra,15599.559956,0.954119
4,NaN,NaN,NaN,Angola,2021-04-02 04:20:36,-11.20270,17.873900,22399,538,20508.0,1353.0,Angola,68.151942,2.401893


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,56177,56192,56226,56254,56290,56294,56322,56384,56454,56517
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,121847,122295,122767,123216,123641,124134,124419,124723,125157,125506
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,116349,116438,116543,116657,116750,116836,116946,117061,117192,117304


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2466,2466,2467,2467,2469,2470,2472,2476,2484,2489
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2156,2171,2184,2192,2204,2210,2216,2227,2235,2241
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3066,3069,3071,3074,3077,3080,3084,3089,3093,3096


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,49752,49802,49937,49994,49994,50013,50666,51473,51550,51788
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,86497,87193,87760,88349,88899,89456,90024,90617,91271,91875
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,80981,81065,81065,81160,81242,81316,81349,81442,81538,81632


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6525,6533,6540,6543,6562,6570,6577,6580,6589,6595
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,20354,20395,20417,20423,20453,20473,20487,20492,20505,20523
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2213,2216,2218,2221,2224,2226,2226,2227,2227,2227


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,99,99,99,99,99,99,99,99,99,99
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,297,300,300,301,301,301,301,301,301,301
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,54,54,54,54,54,54,54,54,55,55


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-04-02 04:20:36,32.539527,-86.644082,6595,99,NaN,NaN,"Autauga, Alabama, US",11804.399578,1.501137
689,1075.0,Lamar,Alabama,US,2021-04-02 04:20:36,33.779950,-88.096680,1390,33,NaN,NaN,"Lamar, Alabama, US",10068.815647,2.374101
690,1077.0,Lauderdale,Alabama,US,2021-04-02 04:20:36,34.901719,-87.656247,9144,219,NaN,NaN,"Lauderdale, Alabama, US",9860.992785,2.395013


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,30539175,553120,0.0
Brazil,12839844,325284,11240213.0
India,12221665,162927,11474683.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,30539175,553120,0.0,29986055.0,2021-04-02 04:20:36,37.936303,-91.379001
Brazil,12839844,325284,11240213.0,1274347.0,2021-04-02 04:20:36,-12.669522,-48.480493
India,12221665,162927,11474683.0,584055.0,2021-04-02 04:20:36,23.088275,81.806127
France,4755779,96106,302995.0,4356678.0,2021-04-02 04:20:36,6.430808,-34.730285
Russia,4503291,97594,4128769.0,276928.0,2021-04-02 04:20:36,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3670765,59395,0.0
Texas,2797010,48495,0.0
Florida,2064525,33494,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3670765,59395,0.0,3611370.0,2021-04-02 04:20:36,37.843962,-120.728594
Texas,2797010,48495,0.0,2748515.0,2021-04-02 04:20:36,31.660643,-98.653069
Florida,2064525,33494,0.0,2031031.0,2021-04-02 04:20:36,28.940755,-82.700744
New York,1882308,50375,0.0,1831933.0,2021-04-02 04:20:36,42.544151,-75.474183
Illinois,1248021,23601,0.0,1224420.0,2021-04-02 04:20:36,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1220246,23189,0.0
Arizona,Maricopa,524547,9677,0.0
Illinois,Cook,499429,9672,0.0
Florida,Miami-Dade,447042,5878,0.0
Texas,Harris,377720,5826,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1220246,23189,0.0,1197057.0,2021-04-02 04:20:36,34.308284,-118.228241,6037.0
Arizona,Maricopa,524547,9677,0.0,514870.0,2021-04-02 04:20:36,33.348359,-112.491815,4013.0
Illinois,Cook,499429,9672,0.0,489757.0,2021-04-02 04:20:36,41.841448,-87.816588,17031.0
Florida,Miami-Dade,447042,5878,0.0,441164.0,2021-04-02 04:20:36,25.611236,-80.551706,12086.0
Texas,Harris,377720,5826,0.0,371894.0,2021-04-02 04:20:36,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-30,56384,124723,117061,11944,22182,1136,2332765,191491,29304,542542,...,4355867,102461,82682,3,159149,2594,240065,4247,88199,36839
2021-03-31,56454,125157,117192,12010,22311,1136,2348821,192639,29322,546229,...,4359982,105549,82869,3,160497,2603,242353,4357,88418,36882
2021-04-01,56517,125506,117304,12053,22399,1147,2363251,193736,29333,549592,...,4364547,108188,83050,3,161751,2617,244645,4531,88549,36896


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-30,2476,2227,3089,115,536,28,55736,3497,909,9308,...,126912,953,628,0,1589,35,2614,882,1202,1520
2021-03-31,2484,2235,3093,115,537,28,55858,3515,909,9339,...,126955,974,629,0,1602,35,2627,888,1208,1523
2021-04-01,2489,2241,3096,115,538,28,55941,3533,909,9368,...,127006,1009,630,0,1615,35,2645,906,1212,1523


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-30,51473,90617,81442,11276,20446,820,2095853,172605,23005,497816,...,12930,81438,80799,1,147163,2359,213590,1667,84519,34617
2021-03-31,51550,91271,81538,11315,20493,820,2102616,173064,23012,501116,...,13073,83380,80983,1,147846,2359,215429,1676,84592,34686
2021-04-01,51788,91875,81632,11365,20508,857,2112266,173681,23014,503729,...,13089,85200,81135,1,148656,2359,217224,1682,84698,34698


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6525,6533,6540,6543,6562,6570,6577,6580,6589,6595
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1372,1378,1378,1382,1384,1384,1384,1384,1384,1390
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9034,9065,9070,9076,9089,9100,9102,9116,9140,9144
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2910,2911,2912,2914,2917,2918,2922,2926,2926,2928
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,15292,15333,15355,15371,15381,15389,15394,15397,15417,15433


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-30,514980,62797,0,841078,330186,3666266,460318,310056,94573,49,...,810529,2789428,385127,19109,2907,617941,363235,141322,635314,56236
2021-03-31,515388,62797,0,841811,330398,3668745,462081,310624,94802,49,...,811842,2791910,385641,19275,2907,618976,364486,141738,636041,56310
2021-04-01,515866,63189,0,842192,330609,3670765,463685,312468,95188,49,...,813614,2797010,386128,19523,2907,620801,365762,142233,636908,56389


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-03-30        6580   20492    2227  2537   6443    1206   2098   14216   
2021-03-31        6589   20505    2227  2542   6444    1207   2097   14224   
2021-04-01        6595   20523    2227  2543   6446    1207   2103   14227   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-03-30         3487     1806  ...    2642    610     3078      703   
2021-03-31         3488     1806  ...    2640    610     3079      705   
2021-04-01         3489     1810  ...    2640    610     3080      707   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-03-30           4006  3593  2125          0      891    632  
2021-03-31           4022  3609  2128          0      891    633  
2021-04-01           4030  3618  2134          0      891    634  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-30,10527,313,0,16941,5616,59128,6099,7885,1559,0,...,11894,48242,2118,225,26,10242,5237,2640,7292,695
2021-03-31,10554,313,0,16967,5626,59274,6107,7889,1559,0,...,11904,48252,2122,227,26,10252,5247,2676,7315,700
2021-04-01,10553,313,0,16977,5636,59395,6117,7900,1559,0,...,11915,48495,2125,227,26,10268,5262,2683,7326,700


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-03-30          99     301      54   58    131      39     66     304   
2021-03-31          99     301      55   58    131      40     66     305   
2021-04-01          99     301      55   58    131      40     66     305   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-03-30          116       43  ...      30     11       31        7   
2021-03-31          117       43  ...      30     11       31        7   
2021-04-01          117       43  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-03-30             36     9    12          0       26      5  
2021-03-31             37     9    12          0       26      5  
2021-04-01             37     9    12          0       26      5  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-30,0.001101,0.002443,0.000983,0.004711,0.002259,0.000000,0.004372,0.003932,0.000273,0.005562,...,0.000935,0.028890,0.001769,0.0,0.007636,0.000000,0.007627,0.032078,0.002125,0.000000
2021-03-31,0.001241,0.003480,0.001119,0.005526,0.005816,0.000000,0.006883,0.005995,0.000614,0.006796,...,0.000945,0.030138,0.002262,0.0,0.008470,0.003470,0.009531,0.025901,0.002483,0.001167
2021-04-01,0.001116,0.002788,0.000956,0.003580,0.003944,0.009683,0.006144,0.005695,0.000375,0.006157,...,0.001047,0.025003,0.002184,0.0,0.007813,0.005378,0.009457,0.039936,0.001482,0.000380


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-30,0.001618,0.004964,0.001621,0.0,0.003745,0.0,0.002248,0.006041,0.0,0.003558,...,0.000434,0.026940,0.003195,NaN,0.007609,0.0,0.005771,0.022016,0.001667,0.000000
2021-03-31,0.003231,0.003592,0.001295,0.0,0.001866,0.0,0.002189,0.005147,0.0,0.003330,...,0.000339,0.022036,0.001592,NaN,0.008181,0.0,0.004973,0.006803,0.004992,0.001974
2021-04-01,0.002013,0.002685,0.000970,0.0,0.001862,0.0,0.001486,0.005121,0.0,0.003105,...,0.000402,0.035934,0.001590,NaN,0.008115,0.0,0.006852,0.020270,0.003311,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-30,0.015928,0.006587,0.001143,0.002846,0.003238,0.000000,0.003638,0.002958,0.000000,0.006956,...,0.004818,0.025538,0.001636,0.0,0.006614,0.022097,0.009782,0.006035,0.000651,0.000000
2021-03-31,0.001496,0.007217,0.001179,0.003459,0.002299,0.000000,0.003227,0.002659,0.000304,0.006629,...,0.011060,0.023846,0.002277,0.0,0.004641,0.000000,0.008610,0.005399,0.000864,0.001993
2021-04-01,0.004617,0.006618,0.001153,0.004419,0.000732,0.045122,0.004590,0.003565,0.000087,0.005214,...,0.001224,0.021828,0.001877,0.0,0.005479,0.000000,0.008332,0.003580,0.001253,0.000346


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-30,0.000701,0.010231,NaN,0.000697,0.000539,0.000642,0.002083,0.005243,0.001811,0.0,...,0.001034,0.001152,0.000964,0.005631,0.0,0.002323,0.002346,0.002348,0.001357,0.000819
2021-03-31,0.000792,0.000000,NaN,0.000872,0.000642,0.000676,0.003830,0.001832,0.002421,0.0,...,0.001620,0.000890,0.001335,0.008687,0.0,0.001675,0.003444,0.002944,0.001144,0.001316
2021-04-01,0.000927,0.006242,NaN,0.000453,0.000639,0.000551,0.003471,0.005936,0.004072,0.0,...,0.002183,0.001827,0.001263,0.012866,0.0,0.002948,0.003501,0.003492,0.001363,0.001403


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-03-30      0.000456  0.000244  0.000449  0.000394  0.002646  0.001661   
2021-03-31      0.001368  0.000634  0.000000  0.001971  0.000155  0.000829   
2021-04-01      0.000911  0.000878  0.000000  0.000393  0.000310  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-03-30      0.000477  0.000704  0.000574  0.001109  ... -0.000756   
2021-03-31     -0.000477  0.000563  0.000287  0.000000  ... -0.000757   
2021-04-01      0.002861  0.000211  0.000287  0.002215  ...  0.000000   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-03-30      0.003289  0.001301  0.004286   0.001000  0.000000  0.000942   
2021-03-31      0.000000  0.000325  0.002845   0.003994  0.004453  0.001412   
2021-04-01      0.000000  0.000325  0.002837   0.001989  0.002494  0.002820   

Province_State                                
Admin2         Unassigned Washakie    Weston  
2021-03-30            NaN      0.0  0.000000  
2021-03-31            NaN      0.0  0.001582  
2021-04-01            NaN      0.0  0.001580  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-30,0.000095,0.0,NaN,0.001359,0.002678,0.001660,0.001149,0.000254,0.00451,NaN,...,0.002360,0.002244,0.001892,0.000000,0.0,0.002251,0.002105,0.000758,0.002061,0.000000
2021-03-31,0.002565,0.0,NaN,0.001535,0.001781,0.002469,0.001312,0.000507,0.00000,NaN,...,0.000841,0.000207,0.001889,0.008889,0.0,0.000976,0.001909,0.013636,0.003154,0.007194
2021-04-01,-0.000095,0.0,NaN,0.000589,0.001777,0.002041,0.001637,0.001394,0.00000,NaN,...,0.000924,0.005036,0.001414,0.000000,0.0,0.001561,0.002859,0.002616,0.001504,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                 \
Admin2         Autauga Baldwin   Barbour Bibb Blount   Bullock Butler   
2021-03-30         0.0     0.0  0.000000  0.0    0.0  0.000000    0.0   
2021-03-31         0.0     0.0  0.018519  0.0    0.0  0.025641    0.0   
2021-04-01         0.0     0.0  0.000000  0.0    0.0  0.000000    0.0   

Province_State                               ... Wyoming                  \
Admin2           Calhoun  Chambers Cherokee  ...    Park Platte Sheridan   
2021-03-30      0.000000  0.000000      0.0  ...     0.0    0.0      0.0   
2021-03-31      0.003289  0.008621      0.0  ...     0.0    0.0      0.0   
2021-04-01      0.000000  0.000000      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-03-30          0.0   0.000000   0.0   0.0        NaN      0.0    0.0  
2021-03-31          0.0   0.027778   0.0   0.0        NaN      0.0    0.0  
2021-04-01          0.0   0.000000   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-30,0.000755,0.002738,0.000926,0.004246,0.002447,0.003430,0.004667,0.003751,0.000418,0.005675,...,0.001000,0.026464,0.002014,2.384186e-07,0.007291,0.000640,0.007415,0.025832,0.002063,0.000198
2021-03-31,0.000998,0.003109,0.001023,0.004886,0.004131,0.001715,0.005775,0.004873,0.000516,0.006235,...,0.000973,0.028301,0.002138,1.192093e-07,0.007881,0.002055,0.008473,0.025866,0.002273,0.000683
2021-04-01,0.001057,0.002949,0.000989,0.004233,0.004038,0.005699,0.005959,0.005284,0.000446,0.006196,...,0.001010,0.026652,0.002161,5.960464e-08,0.007847,0.003717,0.008965,0.032901,0.001877,0.000531


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-30,0.001124,0.004141,0.001377,0.000617,0.002505,0.000019,0.002345,0.005009,5.560253e-32,0.003011,...,0.000348,0.022068,0.001929,NaN,0.007448,4.833810e-65,0.006766,0.019183,0.001413,0.000153
2021-03-31,0.002177,0.003867,0.001336,0.000309,0.002185,0.000009,0.002267,0.005078,2.780127e-32,0.003171,...,0.000343,0.022052,0.001761,NaN,0.007814,2.416905e-65,0.005870,0.012993,0.003202,0.001064
2021-04-01,0.002095,0.003276,0.001153,0.000154,0.002024,0.000005,0.001876,0.005099,1.390063e-32,0.003138,...,0.000372,0.028993,0.001675,NaN,0.007965,1.208452e-65,0.006361,0.016632,0.003257,0.000532


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-30,0.011364,0.006487,0.000907,0.002997,0.003062,0.013900,0.003767,0.003189,0.000070,0.006276,...,0.005870,0.023839,0.001677,0.0,0.006371,0.012357,0.008350,0.012607,0.001103,0.000284
2021-03-31,0.006430,0.006852,0.001043,0.003228,0.002680,0.006950,0.003497,0.002924,0.000187,0.006452,...,0.008465,0.023842,0.001977,0.0,0.005506,0.006179,0.008480,0.009003,0.000983,0.001139
2021-04-01,0.005523,0.006735,0.001098,0.003823,0.001706,0.026036,0.004043,0.003245,0.000137,0.005833,...,0.004844,0.022835,0.001927,0.0,0.005492,0.003089,0.008406,0.006292,0.001118,0.000742


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-30,0.000656,0.005297,NaN,0.000703,0.000430,0.000696,0.002104,0.005589,0.002711,2.764737e-115,...,0.001504,0.001224,0.000887,0.007717,0.000973,0.002272,0.002481,0.002565,0.001051,0.001154
2021-03-31,0.000724,0.002648,NaN,0.000787,0.000536,0.000686,0.002967,0.003711,0.002566,1.382368e-115,...,0.001562,0.001057,0.001111,0.008202,0.000486,0.001973,0.002962,0.002754,0.001098,0.001235
2021-04-01,0.000826,0.004445,NaN,0.000620,0.000587,0.000618,0.003219,0.004823,0.003319,6.911842e-116,...,0.001872,0.001442,0.001187,0.010534,0.000243,0.002461,0.003232,0.003123,0.001230,0.001319


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-03-30      0.000877  0.000551  0.000491  0.000272  0.001596  0.001172   
2021-03-31      0.001122  0.000593  0.000246  0.001122  0.000875  0.001001   
2021-04-01      0.001016  0.000735  0.000123  0.000757  0.000593  0.000500   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-03-30      0.000847  0.000540  0.000441  0.000733  ... -0.000259   
2021-03-31      0.000185  0.000551  0.000364  0.000367  ... -0.000508   
2021-04-01      0.001523  0.000381  0.000325  0.001291  ... -0.000254   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-03-30      0.002086  0.001554  0.004763   0.001528  0.002347  0.001034   
2021-03-31      0.001043  0.000940  0.003804   0.002761  0.003400  0.001223   
2021-04-01      0.000522  0.000632  0.003320   0.002375  0.002947  0.002021   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-03-30           -1.0  0.000290  0.000006  
2021-03-31           -1.0  0.000145  0.000794  
2021-04-01           -1.0  0.000072  0.001187  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-30,0.000182,0.000077,NaN,0.000886,0.001920,0.001923,0.000707,0.000743,0.003903,NaN,...,0.002339,0.001762,0.002160,0.000863,0.001250,0.002072,0.001532,0.001080,0.001212,0.000011
2021-03-31,0.001373,0.000038,NaN,0.001210,0.001850,0.002196,0.001010,0.000625,0.001952,NaN,...,0.001590,0.000985,0.002024,0.004876,0.000625,0.001524,0.001721,0.007358,0.002183,0.003603
2021-04-01,0.000639,0.000019,NaN,0.000900,0.001814,0.002119,0.001323,0.001010,0.000976,NaN,...,0.001257,0.003010,0.001719,0.002438,0.000313,0.001542,0.002290,0.004987,0.001843,0.001801


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin   Barbour          Bibb    Blount   
2021-03-30      0.000043  0.000197  0.000001 -4.954515e-10  0.000061   
2021-03-31      0.000022  0.000098  0.009260 -2.477258e-10  0.000031   
2021-04-01      0.000011  0.000049  0.004630 -1.238629e-10  0.000015   

Province_State                                                            ...  \
Admin2               Bullock        Butler   Calhoun  Chambers  Cherokee  ...   
2021-03-30      2.579814e-08  3.809442e-12  0.000014  0.000206  0.000372  ...   
2021-03-31      1.282053e-02  1.904721e-12  0.001652  0.004413  0.000186  ...   
2021-04-01      6.410263e-03  9.523605e-13  0.000826  0.002207  0.000093  ...   

Province_State   Wyoming                                            \
Admin2              Park        Platte      Sheridan      Sublette   
2021-03-30      0.000135  5.468071e-21  1.017460e-06  4.469791e-27   
2021-03-31      0.000067  2.734036e-21  5.087298e-07  2.234896e-27   
2021-04-01      0.000034  1.367018e-21  2.543649e-07  1.117448e-27   

Province_State                                                       \
Admin2            Sweetwater         Teton         Uinta Unassigned   
2021-03-30      1.612206e-10  1.426841e-14  7.251121e-22  -0.501961   
2021-03-31      1.388889e-02  7.134207e-15  3.625561e-22  -0.501961   
2021-04-01      6.944444e-03  3.567104e-15  1.812780e-22  -0.501961   

Province_State                              
Admin2              Washakie        Weston  
2021-03-30      2.777157e-19  5.898060e-18  
2021-03-31      1.388578e-19  2.949030e-18  
2021-04-01      6.942892e-20  1.474515e-18  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
